In [1]:

import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras.utils import pad_sequences
import keras
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import pickle
from keras.callbacks import EarlyStopping
from sklearn.utils import shuffle


In [2]:
file = open('data_pickles/train_test_val_Human_posTags', 'rb')
X_human_train_posTags, X_human_test_posTags, X_human_val_posTags = pickle.load(file)
file.close()

Y_human_train = np.zeros(len(X_human_train_posTags))
Y_human_test = np.zeros(len(X_human_test_posTags))
Y_human_val = np.zeros(len(X_human_val_posTags))


file = open('data_pickles/train_test_val_GPT_posTags', 'rb')
X_gpt_train_posTags, X_gpt_test_posTags, X_gpt_val_posTags = pickle.load(file)
file.close()

Y_gpt_train = np.ones(len(X_gpt_train_posTags))
Y_gpt_test = np.ones(len(X_gpt_test_posTags))
Y_gpt_val = np.ones(len(X_gpt_val_posTags))

In [3]:
len(X_human_train_posTags), len(Y_human_train), len(X_human_test_posTags), len(Y_human_test), len(X_human_val_posTags), len(Y_human_val)

(435, 435, 95, 95, 93, 93)

In [ ]:
X_human_train_posTags[0]

In [5]:
X_train = np.hstack((X_human_train_posTags, X_gpt_train_posTags))
Y_train = np.hstack((Y_human_train, Y_gpt_train))

X_test = np.hstack((X_human_test_posTags, X_gpt_test_posTags))
Y_test = np.hstack((Y_human_test, Y_gpt_test))

# Juntando train e test
# # ------------------------------------------------------
# X_train = np.hstack((X_train, X_test))
# Y_train = np.hstack((Y_train, Y_test))
# # ------------------------------------------------------


X_val = np.hstack((X_human_val_posTags, X_gpt_val_posTags))
Y_val = np.hstack((Y_human_val, Y_gpt_val))

full_dataset = np.hstack((X_train, X_test, X_val))

In [6]:
X_train = [' '.join(data) for data in X_train]
X_test = [' '.join(data) for data in X_test]
X_val = [' '.join(data) for data in X_val]

full_dataset = [' '.join(data) for data in full_dataset]


In [ ]:
# https://www.analyticsvidhya.com/blog/2022/01/sentiment-analysis-with-lstm/
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/utils/pad_sequences
# https://keras.io/api/layers/core_layers/embedding/

In [7]:
# 226 = tipos de postags possiveis, talvez trocar para somente as que aparecem nos arquivos
# vocab_size = 226 # choose based on statistics

# 93%
# vocab_size = 50
# oov_tok = ''
# embedding_dim = 200
# max_length = 200

vocab_size = 55
oov_tok = ''
embedding_dim = 150
max_length = 200



# max_length = 500

padding_type='post'
trunc_type='post'

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)

# tokenizer.fit_on_texts(X_train)
tokenizer.fit_on_texts(full_dataset)
word_index = tokenizer.word_index


train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)

test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

val_sequences = tokenizer.texts_to_sequences(X_val)
val_padded = pad_sequences(val_sequences, padding='post', maxlen=max_length)


In [8]:
train_padded, Y_train = shuffle(train_padded, Y_train)
test_padded, Y_test = shuffle(test_padded, Y_test)
val_padded, Y_val = shuffle(val_padded, Y_val)

In [9]:
lstm_dataset_test= [test_padded, Y_test]

filename = 'lstm_dataset_test'
file = open(filename, 'wb')
pickle.dump(lstm_dataset_test, file)
file.close()

In [8]:
train_padded[0]

array([ 5, 16,  4,  2,  3,  9,  7,  2, 14,  7,  3,  6,  2,  5, 23, 10, 10,
        3,  7,  3,  5,  3,  5,  4,  2, 16,  5,  3,  9, 16,  2, 16,  4,  2,
       10,  3,  5, 13,  8,  4,  2,  9, 10, 16,  3,  7, 16, 10, 15,  6, 13,
        2,  4,  6,  7,  5, 21,  6,  2, 16,  3, 29, 19,  8,  4,  6,  2, 23,
       10, 11,  4,  2, 12, 15,  2,  3,  7, 26, 17,  9,  6,  2, 19,  8, 15,
        3,  4,  2,  4,  2, 16, 13,  8,  4,  5, 12,  2, 13,  8,  7,  4,  2,
       21,  2,  2, 12,  8,  3,  6,  7,  6,  3,  7, 11,  7, 19,  8, 15, 13,
        8, 14,  4,  2, 11,  3,  2, 12, 10, 15,  3, 10,  6,  2, 12, 15, 15,
        4,  2, 10, 12, 10,  4,  2,  3,  5, 16,  9,  6,  5, 13,  4,  2,  3,
        2,  7, 11,  7,  5, 11,  5,  4,  6,  2,  3,  5, 17, 15, 14,  3,  2,
        3,  4, 14,  2,  3,  6,  2,  7, 11,  4,  2,  3,  6,  2,  7, 11,  7,
       12,  6, 15,  7,  3,  4, 19,  8,  6, 11, 17, 15,  7])

In [9]:
train_padded.shape, val_padded.shape

((870, 200), (186, 200))

In [36]:
# vocab_size = 50
# oov_tok = ''

# tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
# tokenizer.fit_on_texts(X_train)
# word_index = tokenizer.word_index

# embedding_dim = 100
# padding_type='post'


# train_sequences = tokenizer.texts_to_sequences(X_train)
# train_padded = pad_sequences(train_sequences, padding='post')

# test_sequences = tokenizer.texts_to_sequences(X_test)
# test_padded = pad_sequences(test_sequences, padding='post')

# val_sequences = tokenizer.texts_to_sequences(X_val)
# val_padded = pad_sequences(val_sequences, padding='post')

In [10]:
# model = keras.Sequential([
#     keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
#     keras.layers.Bidirectional(keras.layers.LSTM(128)),
#     keras.layers.Dense(32, activation='relu'),
#     keras.layers.Dense(1, activation='sigmoid')
# ])

# 93%
# model = keras.Sequential([
#     keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
#     keras.layers.Bidirectional(keras.layers.LSTM(64)),
#     # keras.layers.Dense(64, activation='relu'),

#     keras.layers.Dense(32, activation='relu'),
#     keras.layers.Dense(1, activation='sigmoid')
# ])

# ESSE
# model = keras.Sequential([
#     keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
#     keras.layers.Bidirectional(keras.layers.LSTM(64)),
#     # keras.layers.Dense(32, activation='relu'),
#     keras.layers.Dropout(0.2),
#     keras.layers.Dense(32, activation='relu'),
#     keras.layers.Dense(1, activation='sigmoid')
# ])

# model = keras.Sequential([
#     keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim),
#     keras.layers.Bidirectional(keras.layers.LSTM(64)),
#     keras.layers.Dense(24, activation='relu'),
#     keras.layers.Dense(1, activation='sigmoid')
# ])

# 9402_9086_9157
# model = keras.Sequential([
#     keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
#     # keras.layers.SpatialDropout1D(0.2),
#     keras.layers.Bidirectional(keras.layers.LSTM(64, dropout=0.2, recurrent_dropout=0.2)),
#     # keras.layers.Dense(32, activation='relu'),
#     # keras.layers.Dropout(0.2),
#     keras.layers.Dense(32, activation='relu'),
#     keras.layers.Dense(1, activation='sigmoid')
# ])

model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64, dropout=0.2, recurrent_dropout=0.2)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])


model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 150)          8250      
                                                                 
 bidirectional (Bidirectiona  (None, 128)              110080    
 l)                                                              
                                                                 
 dense (Dense)               (None, 32)                4128      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 122,491
Trainable params: 122,491
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

callback = EarlyStopping(monitor='accuracy', patience=5)

In [12]:
num_epochs = 27
history = model.fit(train_padded, Y_train, 
                    epochs=num_epochs, 
                    verbose=1, 
                    shuffle=True,
                    validation_data=(val_padded, Y_val),
                    callbacks=[callback])

Epoch 1/27
28/28 [==============================] - 32s 927ms/step - loss: 0.6285 - accuracy: 0.6851 - val_loss: 0.5491 - val_accuracy: 0.7151
Epoch 2/27
28/28 [==============================] - 24s 857ms/step - loss: 0.3750 - accuracy: 0.8563 - val_loss: 0.3151 - val_accuracy: 0.8656
Epoch 3/27
28/28 [==============================] - 25s 892ms/step - loss: 0.2477 - accuracy: 0.9103 - val_loss: 0.3337 - val_accuracy: 0.8925
Epoch 4/27
28/28 [==============================] - 23s 812ms/step - loss: 0.2340 - accuracy: 0.9115 - val_loss: 0.2417 - val_accuracy: 0.8978
Epoch 5/27
28/28 [==============================] - 23s 818ms/step - loss: 0.2438 - accuracy: 0.9092 - val_loss: 0.3282 - val_accuracy: 0.8817
Epoch 6/27
28/28 [==============================] - 23s 811ms/step - loss: 0.2181 - accuracy: 0.9207 - val_loss: 0.3895 - val_accuracy: 0.8710
Epoch 7/27
28/28 [==============================] - 23s 818ms/step - loss: 0.2563 - accuracy: 0.9092 - val_loss: 0.4329 - val_accuracy: 0.8548

In [14]:
model.save('modelo_9437_9140_9157.h5')

In [12]:
num_epochs = 25
history = model.fit(train_padded, Y_train, 
                    epochs=num_epochs, 
                    verbose=1, 
                    #validation_split=0.1,
                    validation_data=(val_padded, Y_val),
                    callbacks=[callback])

Epoch 1/25
24/24 [==============================] - 93s 4s/step - loss: 0.6468 - accuracy: 0.6114 - val_loss: 0.5354 - val_accuracy: 0.7979
Epoch 2/25
24/24 [==============================] - 78s 3s/step - loss: 0.5141 - accuracy: 0.7586 - val_loss: 0.3648 - val_accuracy: 0.8564
Epoch 3/25
24/24 [==============================] - 64s 3s/step - loss: 0.3412 - accuracy: 0.8554 - val_loss: 0.2650 - val_accuracy: 0.9043
Epoch 4/25
24/24 [==============================] - 63s 3s/step - loss: 0.3493 - accuracy: 0.8475 - val_loss: 0.3702 - val_accuracy: 0.8404
Epoch 5/25
24/24 [==============================] - 54s 2s/step - loss: 0.2925 - accuracy: 0.8780 - val_loss: 0.2175 - val_accuracy: 0.9202
Epoch 6/25
24/24 [==============================] - 52s 2s/step - loss: 0.2331 - accuracy: 0.9072 - val_loss: 0.2396 - val_accuracy: 0.8777
Epoch 7/25
24/24 [==============================] - 57s 2s/step - loss: 0.2661 - accuracy: 0.8873 - val_loss: 0.2612 - val_accuracy: 0.8883
Epoch 8/25
24/24 [==

In [13]:
score = model.evaluate(test_padded, Y_test, verbose=0)
print('Evaluation loss:', score[0])
print('Evaluation accuracy:', score[1])

Evaluation loss: 0.3139588236808777
Evaluation accuracy: 0.9157894849777222
